In [1]:
import pymysql
import pandas as pd
import numpy as np

In [3]:
# 데이터베이스 접속 정보 설정
connection = pymysql.connect(
    host='ahnbi1.suwon.ac.kr',
    user='mimic4_mon_5_reader',
    password='suwondatabase',
    db='mimic4_fri_7'
)

try:
    with connection.cursor() as cursor:
        # SQL 쿼리 작성
        query = """
            WITH drugs_of_patients AS(
                SELECT subject_id, GROUP_CONCAT(drug) AS drugs
                FROM hosp_prescriptions
                GROUP BY subject_id
            )
            SELECT icd_code, drugs
            FROM hosp_diagnoses_icd A JOIN drugs_of_patients B ON A.subject_id = B.subject_id
            WHERE icd_version = 10
            GROUP BY icd_code;
        """
        
        # 쿼리 실행 및 결과 가져오기
        cursor.execute(query)
        result = cursor.fetchall()
        
        # 결과를 데이터프레임으로 변환
        df = pd.DataFrame(result, columns=['icd_code', 'drugs'])
        
        top_10_drugs = ['Insulin', '0.9% Sodium Chloride', 'Potassium Chloride', 'Sodium Chloride 0.9%  Flush', 'Furosemide', '5% Dextrose', 'Magnesium Sulfate', 'Metoprolol Tartrate', 'Acetaminophen', 'HYDROmorphone']
        
        
        for drugs_name in top_10_drugs:
            print("약물 이름: ", drugs_name)
            
            #drugs_name이 들어간 행들만 추출
            df_drugs = df[df['drugs'].str.contains(drugs_name)]
            
            #icd_code(ICD-10)의 첫글자 알파벳을 뽑아 'code_category'열에 새로 저장
            df_drugs = df_drugs.copy()
            df_drugs.loc[:, 'code_category'] = df_drugs['icd_code'].str[0]
            
            # 'Z'인 행 삭제
            df_drugs = df_drugs[df_drugs['code_category'] != 'Z']
            
            # 상위 3개의 icd_code 종류 출력
            icd_code = df_drugs['code_category'].value_counts().head(3)
            icd_code = pd.DataFrame(icd_code).reset_index()
            icd_code.columns = ['top_3_icdCode', 'count']
            icd_code.index += 1
            print(icd_code)
            print()
            
    

except Exception as e:
    print(f"An error occurred: {e}")
    connection.rollback()  # 오류 발생 시 롤백

finally:
    # 데이터베이스 연결 닫기
    if connection:
        connection.close()

약물 이름:  Insulin
  top_3_icdCode  count
1             I     73
2             R     57
3             E     47

약물 이름:  0.9% Sodium Chloride
  top_3_icdCode  count
1             I     97
2             R     67
3             K     58

약물 이름:  Potassium Chloride
  top_3_icdCode  count
1             I     91
2             R     60
3             K     58

약물 이름:  Sodium Chloride 0.9%  Flush
  top_3_icdCode  count
1             I     97
2             R     67
3             K     58

약물 이름:  Furosemide
  top_3_icdCode  count
1             I     78
2             R     60
3             K     53

약물 이름:  5% Dextrose
  top_3_icdCode  count
1             I     92
2             R     64
3             K     56

약물 이름:  Magnesium Sulfate
  top_3_icdCode  count
1             I     98
2             R     67
3             K     58

약물 이름:  Metoprolol Tartrate
  top_3_icdCode  count
1             I     80
2             R     54
3             K     38

약물 이름:  Acetaminophen
  top_3_icdCode  count
1         

In [4]:
import numpy as np

def classification_of_ICode(x):
    ranges = [
        (0, 2),
        (5, 9),
        (10, 15),
        (20, 25),
        (26, 28),
        (30, 52),
        (60, 69),
        (70, 79),
        (80, 89),
        (95, 99)
    ]
    
    for i, (start, end) in enumerate(ranges):
        if start <= x <= end:
            count_code[i] += 1
            
top_10_drugs = ['Insulin', '0.9% Sodium Chloride', 'Potassium Chloride', 'Sodium Chloride 0.9%  Flush', 'Furosemide', '5% Dextrose', 'Magnesium Sulfate', 'Metoprolol Tartrate', 'Acetaminophen', 'HYDROmorphone']

for drugs_name in top_10_drugs:
    print(drugs_name)
    
    #drugs_name이 들어간 행들만 추출
    df_drugs = df[df['drugs'].str.contains(drugs_name)]
    
    #icd_code(ICD-10)의 첫글자 알파벳을 뽑아 'code_category'열에 새로 저장
    df_drugs = df_drugs.copy()
    df_drugs.loc[:, 'code_category'] = df_drugs['icd_code'].str[0]
    
    #icd_code가 I로 시작하는 행들만 추출
    df_drugs = df_drugs[df_drugs['code_category'] == 'I']
    # 'Z'인 행 삭제
    df_drugs = df_drugs[df_drugs['code_category'] != 'Z']
    #print(df_drugs)
    
    #icd_code 코드 숫자 추출
    I_code_df = df_drugs.copy()
    I_code_df['icd_code'] = I_code_df['icd_code'].str[1:3]
    I_code = I_code_df['icd_code'].astype(int)
    
    count_code = np.zeros(10, dtype=int)
    
    I_code.apply(classification_of_ICode)
    
    print(f'순환 계통 질환 10분류 개수: {count_code}')
    print(f'가장 많이 처방받는 질환 : {count_code.argmax() + 1}분류 질환')
    print()

Insulin
순환 계통 질환 10분류 개수: [ 0  3  4  9  3 26  9  4  7  8]
가장 많이 처방받는 질환 : 6분류 질환

0.9% Sodium Chloride
순환 계통 질환 10분류 개수: [ 0  4  6  9  4 34 13  6 11 10]
가장 많이 처방받는 질환 : 6분류 질환

Potassium Chloride
순환 계통 질환 10분류 개수: [ 0  3  4  9  4 30 13  7 11 10]
가장 많이 처방받는 질환 : 6분류 질환

Sodium Chloride 0.9%  Flush
순환 계통 질환 10분류 개수: [ 0  4  6  9  4 34 13  6 11 10]
가장 많이 처방받는 질환 : 6분류 질환

Furosemide
순환 계통 질환 10분류 개수: [ 0  3  5  8  3 29  9  6  8  7]
가장 많이 처방받는 질환 : 6분류 질환

5% Dextrose
순환 계통 질환 10분류 개수: [ 0  4  6  9  4 33  9  7 11  9]
가장 많이 처방받는 질환 : 6분류 질환

Magnesium Sulfate
순환 계통 질환 10분류 개수: [ 0  4  6  9  4 34 13  7 11 10]
가장 많이 처방받는 질환 : 6분류 질환

Metoprolol Tartrate
순환 계통 질환 10분류 개수: [ 0  4  6  9  2 29 11  7  7  5]
가장 많이 처방받는 질환 : 6분류 질환

Acetaminophen
순환 계통 질환 10분류 개수: [ 0  4  6  9  2 32 13  6  9 10]
가장 많이 처방받는 질환 : 6분류 질환

HYDROmorphone
순환 계통 질환 10분류 개수: [ 0  3  6  6  2 28  3  4  8  3]
가장 많이 처방받는 질환 : 6분류 질환

